# Collaborative Filtering - Movie Recommendation Project
---

## BUSINESS PROBLEM
---
The online movie streaming platform (for example, superperfectmoviesblabla.tv) wants to develop a recommendation system using collaborative filtering. They aim to generate recommendations that encompass the opinions of the company community who have experimented with content-based recommendation systems. When users like a movie, they want to be recommended other movies that have similar likability patterns.

## DATASET STORY
---

The dataset has been provided by MovieLens, containing movies and the ratings given to these movies. The dataset includes approximately 2,000,000 ratings for around 27,000 movies.

## FEATURES
---
## movie.csv
- **movieId:** Unique movie number (UniqueID)
- **title:** Movie title

## rating.csv
- **userId:** Unique user number (UniqueID)
- **movieId:** Unique movie number (UniqueID)
- **rating:** Rating given to the movie by the user
- **timestamp:** Rating date

## STEPS
---
### Step 1: Data Preprocessing
### Step 2: Creating User-Movie DataFrame
### Step 3: Making Item-Based Movie Recommendations
### Step 4: Making User-Based Movie Recommendations
### Step 5: Preparation of Working Script
---

### *Libraries and Settings*
---

In [1]:
############################################################################################################################################
import pandas as pd
import numpy as np
import warnings
############################################################################################################################################
pd.set_option("display.max_columns", 500)
warnings.filterwarnings("ignore")
############################################################################################################################################

---
# Data Preprocessing
---

In [2]:
############################################################################################################################################
movies = pd.read_csv("/kaggle/input/movielens-25m-latest-dataset/movies.csv")
ratings = pd.read_csv("/kaggle/input/movielens-25m-latest-dataset/ratings.csv")
############################################################################################################################################
df = ratings.merge(movies, how="left")
df.head()
############################################################################################################################################

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,1147878820,Underground (1995),Comedy|Drama|War
4,1,899,3.5,1147868510,Singin' in the Rain (1952),Comedy|Musical|Romance


In [3]:
df.shape

(25000095, 6)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 6 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
 4   title      object 
 5   genres     object 
dtypes: float64(1), int64(3), object(2)
memory usage: 1.1+ GB


In [5]:
# convert timestamp to datetime dtype
df["timestamp"] = pd.to_datetime(df["timestamp"], unit= "s").dt.date
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,2006-05-17,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,2006-05-17,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,2006-05-17,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,2006-05-17,Underground (1995),Comedy|Drama|War
4,1,899,3.5,2006-05-17,Singin' in the Rain (1952),Comedy|Musical|Romance


In [6]:
# check for missing values
df.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
title        0
genres       0
dtype: int64

It is important to use memory efficiently while working with matrix structures in recommendation systems which is required a 'User-Movies Matrix' for correlation calculation in the next step.

That's why we define a threshold to eliminate the movies rated at least 1000 times by users and the users rated at least 30 movies, which we think that might not reflect a collaboration with other users and their preferences.

In [7]:
############################################################################################################################################
# count of movies rated by users based on threshold of 1000
title_count = pd.DataFrame(df["movieId"].value_counts(), columns=["count"])
m_less_than_1000 = len(title_count[title_count["count"] < 1000])
m_more_than_1000 = len(title_count[title_count["count"] >= 1000])

print("*"*100)
print("Total Movie Count: ", len(title_count))
print("*"*100)
print("Movie count rated less than 1000 times: ", m_less_than_1000,"(", round((m_less_than_1000 / len(title_count))*100,2), " %)")
print("Movie count rated more than 1000 times: ", m_more_than_1000,"(", round((m_more_than_1000 / len(title_count))*100,2), " %)")
print("*"*100)

############################################################################################################################################
# count of users rated movies based on threshold of 30
user_count = pd.DataFrame(df["userId"].value_counts(), columns=["count"])
u_less_than_30 = len(user_count[user_count["count"] < 30])
u_more_than_30 = len(user_count[user_count["count"] >= 30])

print("*"*100)
print("Total User Count: ", len(user_count))
print("*"*100)
print("User count rated less than 30 times: ", u_less_than_30,"(", round((u_less_than_30 / len(user_count))*100,2), " %)")
print("User count rated more than 30 times: ", u_more_than_30,"(", round((u_more_than_30 / len(user_count))*100,2), " %)")
print("*"*100)
############################################################################################################################################

****************************************************************************************************
Total Movie Count:  59047
****************************************************************************************************
Movie count rated less than 1000 times:  55253 ( 93.57  %)
Movie count rated more than 1000 times:  3794 ( 6.43  %)
****************************************************************************************************
****************************************************************************************************
Total User Count:  162541
****************************************************************************************************
User count rated less than 30 times:  28650 ( 17.63  %)
User count rated more than 30 times:  133891 ( 82.37  %)
****************************************************************************************************


93.56 % of movies are rated less than 1000 and 17.63 users rated at least 30 movies.
We remove them to reduce matrix size for efficiency while keeping most of the information in dataset. 

In [8]:
movies_to_be_removed = title_count[title_count["count"] < 1000].index.to_list()
users_to_be_removed = user_count[user_count["count"] < 30].index.to_list()

rec_df = df[~df["movieId"].isin(movies_to_be_removed)]
rec_df = rec_df[~rec_df["userId"].isin(users_to_be_removed)]

display(rec_df)

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,2006-05-17,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,1,306,3.5,2006-05-17,Three Colors: Red (Trois couleurs: Rouge) (1994),Drama
2,1,307,5.0,2006-05-17,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,1,665,5.0,2006-05-17,Underground (1995),Comedy|Drama|War
4,1,899,3.5,2006-05-17,Singin' in the Rain (1952),Comedy|Musical|Romance
...,...,...,...,...,...,...
25000090,162541,50872,4.5,2009-04-28,Ratatouille (2007),Animation|Children|Drama
25000091,162541,55768,2.5,2009-04-28,Bee Movie (2007),Animation|Comedy
25000092,162541,56176,2.0,2009-04-28,Alvin and the Chipmunks (2007),Children|Comedy
25000093,162541,58559,4.0,2009-04-28,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX


Now, 21.48 Million records are kept out of 25 Million although we reduced column count 93.56% and 17.63% of row count.

In [9]:
before = len(user_count) * len(title_count)
after = len(rec_df["userId"].value_counts()) * len(rec_df["movieId"].value_counts())

print("Matrix Size BEFORE reduction:", len(user_count), "*", len(title_count),"=", before)
print("Matrix Size AFTER reduction:", len(rec_df["userId"].value_counts()), "*", len(rec_df["movieId"].value_counts()),"=", after)
print("Matrix Size Reduction Rate:", round((before - after)/before, 2)*100, "%")
print("Information kept:", round(len(rec_df)/len(df), 2)*100, "%")

Matrix Size BEFORE reduction: 162541 * 59047 = 9597558427
Matrix Size AFTER reduction: 133891 * 3794 = 507982454
Matrix Size Reduction Rate: 95.0 %
Information kept: 86.0 %


In other words, we reduced matrix size by 95% while keeping 86% of information.

---
# Creating User-Movie DataFrame
---

Thus, we can create 'User-Movies Matrix' using pivot table.

In [10]:
user_movie_matrix = rec_df.pivot_table(index="userId", columns="movieId", values="rating")
user_movie_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,39,41,42,43,44,45,46,47,48,50,52,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,76,78,79,81,82,85,86,88,89,92,93,94,95,97,100,101,102,103,104,105,107,110,111,112,117,118,122,123,125,126,132,135,140,141,144,145,147,149,150,151,153,154,155,156,157,158,159,160,161,162,163,164,165,168,169,170,171,172,173,174,175,176,177,179,180,181,185,186,188,191,193,194,195,196,198,199,203,204,205,207,208,213,215,216,218,222,223,224,225,227,229,230,231,232,233,234,235,236,237,239,242,246,247,248,249,252,253,256,257,258,259,260,261,262,265,266,267,270,271,272,273,274,275,276,277,278,280,281,282,288,289,290,292,293,296,299,300,302,303,305,306,307,308,312,313,314,315,316,317,318,319,322,326,327,328,329,330,332,333,334,337,338,339,340,342,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,360,361,362,364,365,366,367,368,...,107141,107348,107406,107771,108188,108190,108727,108729,108932,108945,109374,109487,109578,109673,109848,110102,110127,110501,110553,110730,110882,111113,111360,111362,111364,111443,111622,111659,111743,111759,111781,111921,112138,112171,112175,112183,112290,112370,112552,112556,112623,112804,112852,112897,113345,113348,113378,113573,113741,114180,114662,114795,114935,115122,115149,115210,115569,115617,115713,116161,116797,116799,116823,116897,116977,117176,117444,117529,117533,117590,117881,117895,118696,118900,118997,119141,119145,119155,120466,120635,120799,121231,122882,122886,122890,122892,122896,122898,122900,122902,122904,122906,122910,122912,122914,122916,122918,122920,122922,122924,122926,125916,127098,127108,127202,128360,128838,129354,130073,130490,130520,130634,132046,132480,132796,133419,133771,134130,134170,134368,134393,134853,135133,135137,135143,135436,135536,135567,135569,135861,135887,136018,136020,136562,136864,137857,138036,139385,139644,140110,140174,140247,140267,140715,140956,141866,142448,142488,142997,143355,143385,143859,146656,148626,149334,149406,152017,152077,152081,152970,156387,157296,157699,158238,158783,158872,158966,159093,159415,159817,159858,160080,160271,160438,160718,160980,161131,161582,161634,162082,162350,162414,162578,162600,162606,163134,163645,164179,164909,165549,165551,166024,166461,166528,166534,166635,166643,166946,167746,168248,168250,168252,168254,168326,168366,168492,168612,169864,169984,170705,171011,171763,173145,173291,173941,174055,175303,175569,175661,176101,176371,176419,177593,177615,177763,177765,178061,179401,179817,179819,180031,182715,183611,183837,183869,183897,185029,187541,187593,187595,188301,189203,189333,189713,192385,192389,192803,194448,195159,201773
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.5,5

Let's select a movieId and get first 10 movies correlated with that movie which are similarly rated movieIds with selected movieId. 

In [11]:
# let's take movie with Id 8 as example
selected_movieId = user_movie_matrix.loc[:,8]
correlated_movieIds = user_movie_matrix.corrwith(selected_movieId).sort_values(ascending = False)[:10]
correlated_movieIds

movieId
8         1.000000
90522     0.974316
189203    0.965535
89118     0.919393
69524     0.869783
74228     0.859885
112804    0.856975
162082    0.852198
52694     0.848627
95105     0.834364
dtype: float64

---
# Making Item-Based Movie Recommendations
---

First, we define some function to make searching easier for us.

In [12]:
def find_movie_title_using_id(movieId):
    return rec_df[rec_df["movieId"] == movieId]["title"].values[0]

def find_movie_id_using_name(movie_name):
    return rec_df[rec_df["title"] == movie_name]["movieId"].values[0]

def find_movie_names_with_keyword(seached_movie_name):
    movie_names = list()
    [movie_names.append(name) for name in rec_df["title"] if seached_movie_name.lower() in name.lower() and name not in movie_names]
    return movie_names

Now, we seach for a movie using a key.

In [13]:
seached_key_word = "legend"
movie_full_name = find_movie_names_with_keyword(seached_key_word)
movie_full_name

['Legends of the Fall (1994)',
 'Anchorman: The Legend of Ron Burgundy (2004)',
 'Legend of Zorro, The (2005)',
 'I Am Legend (2007)',
 'Anchorman 2: The Legend Continues (2013)',
 "City Slickers II: The Legend of Curly's Gold (1994)",
 'Urban Legend (1998)',
 'Legend of Drunken Master, The (Jui kuen II) (1994)',
 'Urban Legends: Final Cut (2000)',
 'Legend (1985)',
 'Legend of Bagger Vance, The (2000)',
 "Legend of the Guardians: The Owls of Ga'Hoole (2010)"]

As we have found the full name of the movie (assuming that we seached for 'I Am Legend (2007)'), we get related movieId using defined function.

In [14]:
searched_movie_id = find_movie_id_using_name(movie_full_name[3])
searched_movie_id

56174

After getting movieIds correlated with selected movie, we can find first 10 recommendations.

In [15]:
selected_movieId = user_movie_matrix.loc[:,searched_movie_id]
correlated_movies = user_movie_matrix.corrwith(selected_movieId).sort_values(ascending = False)[1:11] # [1:11] defined as movie itself is in index 0
correlated_movieIds = correlated_movies.index.to_list()
correlated_rate = correlated_movies.to_list()

print("*"*100)
print("'", movie_full_name[3], "' - Recommendation List")
print("*"*100)
for no, movieId, rate in zip(range(1,11),correlated_movieIds, correlated_rate):
    movie_name = find_movie_title_using_id(movieId)
    print(no,"-", movie_name, "-"*(70 - len(movie_name)-len(str(no))), " - ", round(rate*100,2),"%")
print("*"*100)

****************************************************************************************************
' I Am Legend (2007) ' - Recommendation List
****************************************************************************************************
1 - Widows' Peak (1994) --------------------------------------------------  -  68.45 %
2 - I, Robot (2004) ------------------------------------------------------  -  61.91 %
3 - Hancock (2008) -------------------------------------------------------  -  55.55 %
4 - World War Z (2013) ---------------------------------------------------  -  52.76 %
5 - Book of Eli, The (2010) ----------------------------------------------  -  52.02 %
6 - Preacher's Wife, The (1996) ------------------------------------------  -  51.1 %
7 - Shanghai Triad (Yao a yao yao dao waipo qiao) (1995) -----------------  -  49.21 %
8 - Focus (2015) ---------------------------------------------------------  -  48.48 %
9 - Day After Tomorrow, The (2004) -----------------------

---
# Making User-Based Movie Recommendations
---

The purpose of this part is to find the movies that is most probable liked by a selected user.

Let's first remember 'User-Movie Matrix' created previously.

In [16]:
user_movie_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,39,41,42,43,44,45,46,47,48,50,52,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,76,78,79,81,82,85,86,88,89,92,93,94,95,97,100,101,102,103,104,105,107,110,111,112,117,118,122,123,125,126,132,135,140,141,144,145,147,149,150,151,153,154,155,156,157,158,159,160,161,162,163,164,165,168,169,170,171,172,173,174,175,176,177,179,180,181,185,186,188,191,193,194,195,196,198,199,203,204,205,207,208,213,215,216,218,222,223,224,225,227,229,230,231,232,233,234,235,236,237,239,242,246,247,248,249,252,253,256,257,258,259,260,261,262,265,266,267,270,271,272,273,274,275,276,277,278,280,281,282,288,289,290,292,293,296,299,300,302,303,305,306,307,308,312,313,314,315,316,317,318,319,322,326,327,328,329,330,332,333,334,337,338,339,340,342,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,360,361,362,364,365,366,367,368,...,107141,107348,107406,107771,108188,108190,108727,108729,108932,108945,109374,109487,109578,109673,109848,110102,110127,110501,110553,110730,110882,111113,111360,111362,111364,111443,111622,111659,111743,111759,111781,111921,112138,112171,112175,112183,112290,112370,112552,112556,112623,112804,112852,112897,113345,113348,113378,113573,113741,114180,114662,114795,114935,115122,115149,115210,115569,115617,115713,116161,116797,116799,116823,116897,116977,117176,117444,117529,117533,117590,117881,117895,118696,118900,118997,119141,119145,119155,120466,120635,120799,121231,122882,122886,122890,122892,122896,122898,122900,122902,122904,122906,122910,122912,122914,122916,122918,122920,122922,122924,122926,125916,127098,127108,127202,128360,128838,129354,130073,130490,130520,130634,132046,132480,132796,133419,133771,134130,134170,134368,134393,134853,135133,135137,135143,135436,135536,135567,135569,135861,135887,136018,136020,136562,136864,137857,138036,139385,139644,140110,140174,140247,140267,140715,140956,141866,142448,142488,142997,143355,143385,143859,146656,148626,149334,149406,152017,152077,152081,152970,156387,157296,157699,158238,158783,158872,158966,159093,159415,159817,159858,160080,160271,160438,160718,160980,161131,161582,161634,162082,162350,162414,162578,162600,162606,163134,163645,164179,164909,165549,165551,166024,166461,166528,166534,166635,166643,166946,167746,168248,168250,168252,168254,168326,168366,168492,168612,169864,169984,170705,171011,171763,173145,173291,173941,174055,175303,175569,175661,176101,176371,176419,177593,177615,177763,177765,178061,179401,179817,179819,180031,182715,183611,183837,183869,183897,185029,187541,187593,187595,188301,189203,189333,189713,192385,192389,192803,194448,195159,201773
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.5,5

We select a random user and find the movieIds that he/she rated.

In [36]:
selected_userId = user_movie_matrix.sample(1).index[0]

print("*"*100)
print("Selected UserId:", selected_userId)
print("*"*100)

umm_for_selected_userId = user_movie_matrix[user_movie_matrix.index == selected_userId] # 'umm' is used as abbreviation of 'user_movie_matrix'
umm_for_selected_userId

****************************************************************************************************
Selected UserId: 48735
****************************************************************************************************


movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,39,41,42,43,44,45,46,47,48,50,52,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,76,78,79,81,82,85,86,88,89,92,93,94,95,97,100,101,102,103,104,105,107,110,111,112,117,118,122,123,125,126,132,135,140,141,144,145,147,149,150,151,153,154,155,156,157,158,159,160,161,162,163,164,165,168,169,170,171,172,173,174,175,176,177,179,180,181,185,186,188,191,193,194,195,196,198,199,203,204,205,207,208,213,215,216,218,222,223,224,225,227,229,230,231,232,233,234,235,236,237,239,242,246,247,248,249,252,253,256,257,258,259,260,261,262,265,266,267,270,271,272,273,274,275,276,277,278,280,281,282,288,289,290,292,293,296,299,300,302,303,305,306,307,308,312,313,314,315,316,317,318,319,322,326,327,328,329,330,332,333,334,337,338,339,340,342,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,360,361,362,364,365,366,367,368,...,107141,107348,107406,107771,108188,108190,108727,108729,108932,108945,109374,109487,109578,109673,109848,110102,110127,110501,110553,110730,110882,111113,111360,111362,111364,111443,111622,111659,111743,111759,111781,111921,112138,112171,112175,112183,112290,112370,112552,112556,112623,112804,112852,112897,113345,113348,113378,113573,113741,114180,114662,114795,114935,115122,115149,115210,115569,115617,115713,116161,116797,116799,116823,116897,116977,117176,117444,117529,117533,117590,117881,117895,118696,118900,118997,119141,119145,119155,120466,120635,120799,121231,122882,122886,122890,122892,122896,122898,122900,122902,122904,122906,122910,122912,122914,122916,122918,122920,122922,122924,122926,125916,127098,127108,127202,128360,128838,129354,130073,130490,130520,130634,132046,132480,132796,133419,133771,134130,134170,134368,134393,134853,135133,135137,135143,135436,135536,135567,135569,135861,135887,136018,136020,136562,136864,137857,138036,139385,139644,140110,140174,140247,140267,140715,140956,141866,142448,142488,142997,143355,143385,143859,146656,148626,149334,149406,152017,152077,152081,152970,156387,157296,157699,158238,158783,158872,158966,159093,159415,159817,159858,160080,160271,160438,160718,160980,161131,161582,161634,162082,162350,162414,162578,162600,162606,163134,163645,164179,164909,165549,165551,166024,166461,166528,166534,166635,166643,166946,167746,168248,168250,168252,168254,168326,168366,168492,168612,169864,169984,170705,171011,171763,173145,173291,173941,174055,175303,175569,175661,176101,176371,176419,177593,177615,177763,177765,178061,179401,179817,179819,180031,182715,183611,183837,183869,183897,185029,187541,187593,187595,188301,189203,189333,189713,192385,192389,192803,194448,195159,201773
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
48735,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,4.5,NaN,NaN,NaN,2.5,NaN,N

In [37]:
bool_for_selected_userId = umm_for_selected_userId.apply(lambda col: col.notnull(), axis=1)
bool_for_selected_userId

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,39,41,42,43,44,45,46,47,48,50,52,57,58,60,61,62,63,64,65,66,68,69,70,71,72,73,74,76,78,79,81,82,85,86,88,89,92,93,94,95,97,100,101,102,103,104,105,107,110,111,112,117,118,122,123,125,126,132,135,140,141,144,145,147,149,150,151,153,154,155,156,157,158,159,160,161,162,163,164,165,168,169,170,171,172,173,174,175,176,177,179,180,181,185,186,188,191,193,194,195,196,198,199,203,204,205,207,208,213,215,216,218,222,223,224,225,227,229,230,231,232,233,234,235,236,237,239,242,246,247,248,249,252,253,256,257,258,259,260,261,262,265,266,267,270,271,272,273,274,275,276,277,278,280,281,282,288,289,290,292,293,296,299,300,302,303,305,306,307,308,312,313,314,315,316,317,318,319,322,326,327,328,329,330,332,333,334,337,338,339,340,342,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,360,361,362,364,365,366,367,368,...,107141,107348,107406,107771,108188,108190,108727,108729,108932,108945,109374,109487,109578,109673,109848,110102,110127,110501,110553,110730,110882,111113,111360,111362,111364,111443,111622,111659,111743,111759,111781,111921,112138,112171,112175,112183,112290,112370,112552,112556,112623,112804,112852,112897,113345,113348,113378,113573,113741,114180,114662,114795,114935,115122,115149,115210,115569,115617,115713,116161,116797,116799,116823,116897,116977,117176,117444,117529,117533,117590,117881,117895,118696,118900,118997,119141,119145,119155,120466,120635,120799,121231,122882,122886,122890,122892,122896,122898,122900,122902,122904,122906,122910,122912,122914,122916,122918,122920,122922,122924,122926,125916,127098,127108,127202,128360,128838,129354,130073,130490,130520,130634,132046,132480,132796,133419,133771,134130,134170,134368,134393,134853,135133,135137,135143,135436,135536,135567,135569,135861,135887,136018,136020,136562,136864,137857,138036,139385,139644,140110,140174,140247,140267,140715,140956,141866,142448,142488,142997,143355,143385,143859,146656,148626,149334,149406,152017,152077,152081,152970,156387,157296,157699,158238,158783,158872,158966,159093,159415,159817,159858,160080,160271,160438,160718,160980,161131,161582,161634,162082,162350,162414,162578,162600,162606,163134,163645,164179,164909,165549,165551,166024,166461,166528,166534,166635,166643,166946,167746,168248,168250,168252,168254,168326,168366,168492,168612,169864,169984,170705,171011,171763,173145,173291,173941,174055,175303,175569,175661,176101,176371,176419,177593,177615,177763,177765,178061,179401,179817,179819,180031,182715,183611,183837,183869,183897,185029,187541,187593,187595,188301,189203,189333,189713,192385,192389,192803,194448,195159,201773
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
48735,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

In [38]:
movieIds_ratedby_selected_userId = bool_for_selected_userId.iloc[0].loc[lambda movieId: movieId == True]
list_movieIds_ratedby_selected_userId = bool_for_selected_userId.iloc[0].loc[lambda movieId: movieId == True].index.to_list()

print("Movie count rated by UserId '", selected_userId,"' =", len(list_movieIds_ratedby_selected_userId))

Movie count rated by UserId ' 48735 ' = 131


Now, we find all users who rated all movies rated by selected user

In [39]:
allUserIds_and_movies_ratedby_selected_userId = user_movie_matrix.loc[:, list_movieIds_ratedby_selected_userId]
allUserIds_and_movies_ratedby_selected_userId # 180 columns, the same with selected userId 

movieId,2,47,50,110,293,296,303,318,356,364,367,434,480,508,527,551,589,593,750,780,784,788,858,904,1036,1183,1198,1204,1210,1213,1219,1221,1225,1258,1295,1485,1608,1610,1617,1645,1704,1721,1792,1892,2006,2023,2028,2067,2167,2324,2329,2571,2617,2628,2692,2710,2858,2959,3147,3176,3252,3285,3298,3503,3977,4011,4022,4025,4226,4262,4993,4995,5299,5418,5952,5956,5989,5995,6373,6664,6874,7153,7317,7361,7438,8644,8665,30707,31410,32587,34319,44191,44555,48516,48780,49530,49772,52319,58559,59369,64614,68157,71838,72998,73017,74458,79132,81564,84954,88140,91653,92259,96829,99114,103253,105844,106487,106782,106920,109374,109487,111781,112552,115210,115617,116797,122882,122892,122904,134130,134853
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,4.5,4.5,NaN,NaN,2.0,NaN,5.0,NaN,4.0,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,1.5,NaN,5.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,5.0,5.0,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,5.0,NaN,5.0,5.0,NaN,4.0,4.0,NaN,NaN,NaN,2.0,NaN,4.0,NaN,4.0,4.0,NaN,3.5,NaN,NaN,5.0,NaN,NaN,NaN,3.0,NaN,4.0,5.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,4.0,NaN,NaN,5.0,5.0,NaN,NaN,NaN,3.0,NaN,NaN,2.5,4.0,4.0,2.5,4.0,4.5,4.0,4.0,NaN,5.0,4.0,4.0,4.0,NaN,3.0,NaN,4.5,4.0,NaN,4.5,4.0,4.0,4.0,4.0,NaN,4.0,3.5,4.0,NaN,5.0,4.0,3.5,NaN,NaN,5.0,3.0,4.0,4.0,NaN,4.0,3.0,4.0,5.0,4.0,4.0,3.5,NaN,NaN,NaN,4.0,4.0,NaN,3.0,4.5,4.0,4.0,4.0,NaN,5.0,NaN,4.0,4.0,3.5,3.5,NaN,4.0,4.0
4,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,2.5,3.5,4.5,NaN,NaN,NaN,NaN,NaN,4.5,4.0,NaN,NaN,3.5,NaN,3.5,NaN,NaN,NaN,NaN,2.5,NaN,NaN,5.0,NaN,NaN,NaN,4.5,NaN,NaN,3.5,NaN,2.5,3.5,5.0,4.5,3.5,NaN,2.5,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,NaN,4.5,5.0,3.5,4.5,NaN,3.5,NaN,5.0,NaN,3.5,4.0,NaN
5,NaN,5.0,5.0,NaN,NaN,4.0,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,4.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162537,N

In [50]:
perc_threshold_rated_same_movies = 0.7 # the threshold for users who rated at least 70% of the same movies rated by selected user
count_threshold_rated_same_movies = len(list_movieIds_ratedby_selected_userId) * perc_threshold_rated_same_movies
userIds_rated_atleast_X_perc_same_movies_with_selected_userId = allUserIds_and_movies_ratedby_selected_userId[allUserIds_and_movies_ratedby_selected_userId.notnull().sum(axis=1) > count_threshold_rated_same_movies]
userIds_rated_atleast_X_perc_same_movies_with_selected_userId

movieId,2,47,50,110,293,296,303,318,356,364,367,434,480,508,527,551,589,593,750,780,784,788,858,904,1036,1183,1198,1204,1210,1213,1219,1221,1225,1258,1295,1485,1608,1610,1617,1645,1704,1721,1792,1892,2006,2023,2028,2067,2167,2324,2329,2571,2617,2628,2692,2710,2858,2959,3147,3176,3252,3285,3298,3503,3977,4011,4022,4025,4226,4262,4993,4995,5299,5418,5952,5956,5989,5995,6373,6664,6874,7153,7317,7361,7438,8644,8665,30707,31410,32587,34319,44191,44555,48516,48780,49530,49772,52319,58559,59369,64614,68157,71838,72998,73017,74458,79132,81564,84954,88140,91653,92259,96829,99114,103253,105844,106487,106782,106920,109374,109487,111781,112552,115210,115617,116797,122882,122892,122904,134130,134853
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
426,NaN,5.0,4.5,4.5,4.0,4.5,NaN,4.5,4.0,NaN,1.0,NaN,NaN,3.5,4.5,NaN,3.5,5.0,2.0,2.5,NaN,NaN,5.0,4.5,4.5,NaN,4.0,4.0,2.5,3.5,3.5,5.0,3.5,4.0,NaN,3.0,3.0,3.5,3.0,NaN,3.5,3.0,NaN,NaN,NaN,1.0,5.0,NaN,2.5,1.0,4.5,3.5,NaN,1.5,3.5,4.0,5.0,5.0,3.5,2.5,4.0,2.5,1.5,NaN,0.5,2.5,4.0,NaN,3.0,5.0,4.5,3.0,3.5,3.0,4.0,3.0,3.5,1.0,NaN,NaN,3.5,4.0,NaN,3.5,3.0,2.0,3.0,4.0,4.5,3.5,NaN,2.5,4.0,3.5,3.0,2.5,2.5,NaN,4.5,1.0,3.5,4.0,3.0,2.0,NaN,2.0,2.5,NaN,0.5,NaN,NaN,NaN,4.5,3.5,0.5,0.5,0.5,3.5,3.5,NaN,NaN,NaN,1.5,4.5,NaN,2.0,5.0,NaN,NaN,NaN,NaN
541,4.5,5.0,5.0,4.5,5.0,5.0,NaN,5.0,5.0,5.0,5.0,NaN,5.0,NaN,5.0,4.5,5.0,5.0,4.5,NaN,NaN,NaN,5.0,4.5,5.0,NaN,5.0,5.0,5.0,5.0,4.5,5.0,4.5,4.5,NaN,4.5,NaN,NaN,5.0,5.0,4.0,4.5,NaN,NaN,NaN,4.0,5.0,NaN,NaN,4.5,5.0,5.0,4.5,3.0,NaN,NaN,4.0,5.0,5.0,NaN,4.5,NaN,NaN,5.0,NaN,4.5,5.0,NaN,5.0,5.0,5.0,4.5,NaN,5.0,5.0,4.0,4.5,4.5,5.0,NaN,4.5,5.0,NaN,4.5,4.0,NaN,5.0,5.0,NaN,5.0,NaN,5.0,4.0,5.0,4.5,NaN,4.0,NaN,5.0,4.0,5.0,5.0,4.5,4.0,4.5,5.0,5.0,NaN,NaN,NaN,NaN,5.0,5.0,5.0,NaN,NaN,NaN,5.0,4.0,5.0,5.0,4.5,4.5,4.0,4.0,4.0,5.0,3.5,4.5,4.5,5.0
548,4.0,4.0,5.0,4.5,4.0,4.5,3.5,5.0,4.0,5.0,2.5,3.0,4.0,NaN,5.0,NaN,4.5,4.5,NaN,3.5,2.5,1.5,NaN,NaN,5.0,NaN,4.5,NaN,4.0,5.0,NaN,NaN,NaN,4.0,NaN,3.5,3.0,4.0,4.0,NaN,4.0,4.0,3.5,2.5,3.5,NaN,4.0,NaN,3.5,NaN,3.5,4.0,3.5,3.0,5.0,4.0,4.0,4.0,4.0,NaN,NaN,4.0,3.0,NaN,3.0,5.0,4.0,3.0,4.5,3.5,4.0,4.0,NaN,4.0,4.0,4.0,3.5,NaN,3.0,3.5,4.0,4.0,3.0,4.5,4.0,3.5,3.5,NaN,4.5,4.0,3.5,4.0,NaN,3.0,4.5,4.0,NaN,NaN,4.0,3.5,4.0,4.0,3.5,3.5,4.0,3.5,4.0,NaN,3.5,3.0,4.0,NaN,NaN,4.5,3.0,NaN,4.0,4.0,4.5,4.0,3.5,4.0,NaN,4.0,NaN,3.5,4.5,4.0,NaN,NaN,NaN
847,NaN,4.5,4.5,4.0,4.0,4.0,NaN,5.0,3.0,3.5,NaN,NaN,3.5,2.5,4.0,4.0,3.0,3.5,5.0,2.0,NaN,NaN,4.5,4.5,3.5,1.5,4.5,5.0,3.5,3.5,5.0,5.0,4.0,4.5,NaN,3.0,2.5,3.0,4.0,4.0,4.0,2.0,NaN,3.5,3.0,3.0,3.0,2.5,NaN,4.0,3.5,4.0,NaN,2.0,4.0,3.0,2.5,5.0,3.0,3.0,2.5,2.0,2.0,4.0,NaN,3.5,2.0,3.0,4.0,1.0,5.0,3.0,NaN,3.5,4.5,3.5,4.0,5.0,3.0,2.0,3.0,4.0,NaN,4.5,2.0,NaN,3.5,2.5,4.0,NaN,NaN,NaN,4.0,2.5,3.5,3.5,4.0,NaN,2.5,2.0,2.0,3.0,NaN,3.0,3.5,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.5,NaN,3.0,4.0,4.0,3.0,NaN,3.0,NaN,NaN,NaN,4.5,NaN,NaN,3.0,4.0
901,3.5,4.5,4.0,4.0,5.0,5.0,NaN,4.0,4.0,4.5,3.0,NaN,4.0,3.5,4.5,4.0,4.0,4.0,4.0,4.0,3.0,2.5,4.5,4.0,4.0,NaN,4.5,5.0,3.5,4.5,4.0,4.0,NaN,5.0,NaN,3.0,3.5,NaN,3.5,NaN,3.5,3.5,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,4.0,4.0,3.0,3.0,4.0,NaN,5.0,5.0,4.0,NaN,NaN,4.0,NaN,NaN,2.5,4.0,4.0,3.5,4.5,4.5,4.0,4.0,2.5,4.0,4.0,NaN,4.0,4.0,3.0,NaN,4.5,4.0,NaN,4.5,4.5,3.5,4.0,4.5,4.0,4.0,3.5,4.0,NaN,4.0,4.5,4.0,NaN,NaN,4.0,4.0,4.0,4.5,3.5,3.5,3.0,4.0,4.0,NaN,3.5,3.5,NaN,NaN,4.0,4.0,4.0,4.5,3.5,4.0,3.5,4.0,4.0,NaN,5.0,3.5,4.0,3.5,5.0,3.5,3.5,4.0,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [46]:
corr_of_userIds_rated_same_movies_with_selected_userId = userIds_rated_atleast_X_perc_same_movies_with_selected_userId.T.corr().unstack()
corr_of_userIds_rated_same_movies_with_selected_userId

userId  userId
426     426       1.000000
        541       0.428736
        548       0.549978
        847       0.202527
        901       0.420778
                    ...   
162516  162271    0.185880
        162286    0.160912
        162334    0.191996
        162349    0.195723
        162516    1.000000
Length: 938961, dtype: float64

In [47]:
userIds_corr_df = pd.DataFrame(corr_of_userIds_rated_same_movies_with_selected_userId, columns=["corr"])
userIds_corr_df.index.names = ["userId_1","userId_2"]
userIds_corr_df.reset_index(inplace=True)
userIds_corr_df

,userId_1,userId_2,corr
0,426,426,1.000000
1,426,541,0.428736
2,426,548,0.549978
3,426,847,0.202527
4,426,901,0.420778
...,...,...,...
938956,162516,162271,0.185880
938957,162516,162286,0.160912
938958,162516,162334,0.191996
938959,162516,162349,0.195723


In [51]:
corr_threshold = 0.5 # users who have at least 60% correlation with selected user
final_users_corr_df = userIds_corr_df[(userIds_corr_df["userId_1"] == selected_userId) & (userIds_corr_df["corr"] >= corr_threshold)].sort_values(by="corr", ascending=False) # set threshold and sort correlations
final_users_corr_df

,userId_1,userId_2,corr
291000,48735,48735,1.000000
291482,48735,132976,0.594367
291269,48735,95055,0.579613
291548,48735,143879,0.573067
291480,48735,132747,0.550089
291631,48735,155408,0.548828
290909,48735,34164,0.545303
290854,48735,26928,0.541808
291518,48735,139105,0.538599
291262,48735,94154,0.538116


In [62]:
list_users_to_filter = final_users_corr_df["userId_2"].to_list()

final_rec_df = user_movie_matrix.loc[list_users_to_filter,list_movieIds_ratedby_selected_userId]
final_rec_df

movieId,2,47,50,110,293,296,303,318,356,364,367,434,480,508,527,551,589,593,750,780,784,788,858,904,1036,1183,1198,1204,1210,1213,1219,1221,1225,1258,1295,1485,1608,1610,1617,1645,1704,1721,1792,1892,2006,2023,2028,2067,2167,2324,2329,2571,2617,2628,2692,2710,2858,2959,3147,3176,3252,3285,3298,3503,3977,4011,4022,4025,4226,4262,4993,4995,5299,5418,5952,5956,5989,5995,6373,6664,6874,7153,7317,7361,7438,8644,8665,30707,31410,32587,34319,44191,44555,48516,48780,49530,49772,52319,58559,59369,64614,68157,71838,72998,73017,74458,79132,81564,84954,88140,91653,92259,96829,99114,103253,105844,106487,106782,106920,109374,109487,111781,112552,115210,115617,116797,122882,122892,122904,134130,134853
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
48735,3.0,4.0,4.5,4.0,4.5,4.5,2.5,4.5,5.0,5.0,4.5,3.5,3.0,4.5,5.0,4.0,3.0,4.5,4.0,3.0,3.5,3.0,5.0,4.5,3.0,4.0,3.0,5.0,5.0,2.5,4.5,5.0,4.0,4.0,4.0,3.5,3.0,3.5,3.5,4.5,5.0,5.0,3.5,3.5,3.0,5.0,4.5,4.5,3.5,4.5,4.5,5.0,2.5,5.0,4.0,1.0,5.0,5.0,5.0,4.0,4.5,3.5,3.5,4.0,2.5,3.5,5.0,2.5,2.5,5.0,3.5,5.0,2.5,3.5,4.0,4.0,4.0,5.0,3.5,3.0,4.5,3.5,3.5,4.5,4.5,3.5,3.5,3.5,3.5,3.5,3.0,4.0,5.0,3.5,4.5,4.0,4.0,4.5,4.5,3.0,4.5,4.5,3.0,4.5,3.5,4.5,4.5,3.5,2.5,2.0,2.0,4.5,4.5,4.5,3.0,5.0,3.0,4.5,4.0,4.0,4.0,3.5,3.0,3.5,3.0,3.0,1.0,2.5,3.5,3.5,4.0
132976,2.5,4.5,3.5,5.0,4.0,4.5,3.5,4.0,4.0,4.0,1.5,3.0,3.5,3.5,5.0,NaN,NaN,3.5,NaN,NaN,NaN,NaN,5.0,NaN,3.0,2.5,0.5,NaN,NaN,4.0,NaN,5.0,3.0,3.5,NaN,NaN,NaN,2.0,4.0,2.5,3.5,3.5,NaN,1.5,3.0,4.5,3.5,NaN,2.5,3.5,4.0,5.0,NaN,NaN,NaN,NaN,4.0,5.0,4.5,NaN,5.0,3.5,3.0,NaN,0.5,4.5,3.0,1.0,3.0,4.0,4.0,4.0,1.0,0.5,4.0,3.0,4.0,3.5,3.5,NaN,4.5,4.0,1.5,3.5,4.5,3.5,0.5,2.5,NaN,4.0,0.5,5.0,NaN,3.5,4.0,3.0,NaN,NaN,4.0,2.5,NaN,3.5,3.0,2.0,3.5,1.5,3.5,NaN,NaN,0.5,NaN,5.0,NaN,4.0,0.5,3.0,NaN,4.5,NaN,3.5,4.5,1.5,NaN,3.5,3.0,3.5,1.0,0.5,2.0,4.0,NaN
95055,3.0,4.0,4.0,3.5,3.0,4.0,NaN,4.5,4.0,4.0,2.5,NaN,3.5,4.5,4.5,3.5,3.0,4.0,3.5,3.0,2.0,2.0,4.5,NaN,3.0,NaN,3.5,NaN,3.5,3.5,NaN,4.0,NaN,3.5,NaN,2.5,3.0,3.5,NaN,3.5,4.0,3.5,NaN,NaN,2.5,NaN,4.0,NaN,2.5,4.5,4.0,4.0,2.5,3.0,NaN,NaN,4.0,4.0,4.0,NaN,3.5,3.5,NaN,NaN,2.5,3.5,4.0,2.0,4.0,4.0,4.0,4.0,NaN,NaN,4.0,3.5,4.0,4.0,3.0,NaN,3.5,4.0,NaN,4.0,3.5,2.5,NaN,NaN,3.5,3.5,3.0,3.5,NaN,4.0,4.0,3.5,NaN,NaN,3.5,3.0,4.0,4.0,3.5,3.5,3.5,4.0,3.5,2.5,NaN,2.0,NaN,4.0,NaN,3.5,2.5,NaN,2.5,3.5,4.0,3.5,4.0,2.5,4.0,3.0,3.0,3.0,4.0,2.5,3.0,3.5,3.5
143879,2.0,5.0,3.5,4.5,3.5,4.0,2.0,5.0,3.5,4.5,4.0,1.0,3.5,NaN,5.0,4.0,5.0,4.0,NaN,2.0,2.0,2.0,4.0,NaN,3.0,4.0,3.5,NaN,5.0,NaN,NaN,4.0,NaN,3.5,3.5,3.0,1.5,NaN,3.5,3.0,4.0,4.0,NaN,2.5,2.0,3.5,NaN,NaN,2.0,5.0,5.0,4.5,2.5,3.5,NaN,NaN,3.5,5.0,4.5,4.0,5.0,3.0,NaN,NaN,0.5,3.5,4.0,2.0,4.5,4.5,4.5,3.5,0.5,3.0,4.0,3.5,4.0,4.5,3.0,1.5,2.0,4.5,2.0,5.0,3.0,2.0,3.5,NaN,4.0,4.0,3.5,3.5,4.0,3.5,4.0,4.0,1.5,NaN,4.5,2.5,4.0,3.5,NaN,4.5,3.5,4.0,4.5,NaN,NaN,2.0,NaN,4.5,4.0,4.0,3.0,3.5,2.5,3.5,4.0,4.0,4.0,NaN,4.0,3.5,NaN,3.5,3.0,NaN,2.5,4.5,3.5
132747,3.0,4.5,3.5,2.0,4.0,5.0,NaN,4.5,4.0,3.0,4.0,2.0,3.0,NaN,4.0,NaN,1.5,3.5,NaN,3.0,NaN,2.0,4.5,NaN,3.0,2.5,NaN,3.0,4.0,4.0,NaN,4.5,NaN,4.0,NaN,NaN,1.5,2.5,NaN,NaN,3.5,3.0,NaN,NaN,3.5,4.5,3.5,NaN,2.5,3.5,4.0,4.0,1.5,4.0,3.0,NaN,3.0,4.5,3.5,4.0,3.0,3.5,NaN,NaN,2.0,4.0,4.0,2.0,3.5,4.5,4.0,4.5,1.5,3.0,3.5,4.0,5.0,4.0,3.0,NaN,4.5,3.5,3.0,4.5,4.0,2.5,2.5,3.5,3.0,3.5,NaN,3.5,4.0,4.0,4.0,4.0,NaN,NaN,3.0,3.5,3.5,4.5,4.0,3.5,3.5,4.0,4.0,NaN,NaN,1.5,NaN,4.0,NaN,3.5,3.0,3.0,3.5,3.5,NaN,4.0,4.5,2.0,3.5,3.5,NaN,4.5,3.0,2.0,NaN,2.5,NaN
155408,3.5,5.0,5.0,5.0,4.0,5.0,NaN,5.0,5.0,3.0,3.5,NaN,3.0,3.5,5.0,NaN,1.0,4.5,NaN,3.0,3.0,2.0,5.0,4.0,2.0,5.0,3.0,4.0,3.0,5.0,NaN,5.0,4.0,3.0,NaN,3.0,NaN,NaN,5.0,3.0,5.0,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,5.0,5.0,2.0,2.0,4.0,1.0,5.0,5.0,5.0,4.0,4.0,3.5,NaN,NaN,2.0,5.0,5.0,3.0,5.0,NaN,5.0,5.0,4.0,3.0,4.0,4.0,4.0,4.5,3.0,NaN,5.0,4.0,NaN,5.0,5.0,NaN,2.5,4.5,4.0,5.0,3.5,4.0,5.0,4.0,4.0,4.0,NaN,4.5,NaN,3.5,5.0,5.0,NaN,4.0,4.0,3.0,5.0,NaN,3.0,1.0,NaN,3.5,3.0,5.0,NaN,NaN,NaN,NaN,2.0,4.0,4.5,NaN,4.0,NaN,NaN,4.0,Na

---
# Preparation of Working Script
---

---
## I hope you found the analysis insightful and informative!

## Your feedback is greatly appreciated, and I welcome any suggestions for improvement. Feel free to reach out with any questions or comments. 👍🏼
---